# QC/Validation Agent: Genomic Quality Control

## Overview

This notebook implements the **QC/Validation Agent** - the second agent in our WES pipeline automation system.

### Agent Purpose
- **Sample-level QC**: Missingness, heterozygosity, Ti/Tv ratio per sample
- **Variant-level QC**: Missing rate, Hardy-Weinberg equilibrium, MAF distribution
- **Cohort-level QC**: Overall statistics, population structure (via PCA)
- **QC Gates**: Automatic pass/fail based on source-specific thresholds

### QC Metrics

**Sample-level**:
- Missingness rate (proportion of missing genotypes)
- Heterozygosity rate and F coefficient

**Variant-level**:
- Missing rate per variant
- Minor allele frequency (MAF)
- Hardy-Weinberg equilibrium (HWE)

**Cohort-level**:
- Overall Ti/Tv ratio
- Variant type distribution (SNPs vs indels)

## Setup and Imports

In [8]:
import os
import subprocess
import json
import tempfile
import shutil
from typing import Dict, List, Any, Optional
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv

# Google ADK imports
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner

# Load environment variables from .env file
load_dotenv()

# Set up paths
DATA_DIR = Path("data")
TOY_VCF = DATA_DIR / "toy.vcf"
OUTPUT_DIR = Path("qc_output")
OUTPUT_DIR.mkdir(exist_ok=True)

# Get API key from environment
if "GOOGLE_API_KEY" not in os.environ:
    raise ValueError(
        "GOOGLE_API_KEY not found in environment. Please set it in .env file."
    )

# Model selection
LLM_MODEL = "gemini-2.5-flash-lite"

# Plotting style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

print(f"✓ Setup complete")
print(f"✓ Using model: {LLM_MODEL}")
print(f"✓ Test VCF: {TOY_VCF}")
print(f"✓ Output directory: {OUTPUT_DIR}")
print(f"✓ API key loaded from .env")

✓ Setup complete
✓ Using model: gemini-2.5-flash-lite
✓ Test VCF: data/toy.vcf
✓ Output directory: qc_output
✓ API key loaded from .env


## Tool 1: Compute Ti/Tv Statistics

Ti/Tv (transition/transversion) ratio is a key quality metric.
- **Expected for WES**: 2.8-3.2
- **Expected for WGS**: 2.0-2.1

In [9]:
def compute_titv_stats(vcf_path: str) -> dict:
    """
    Compute transition/transversion (Ti/Tv) ratio using bcftools stats.

    Args:
        vcf_path: Path to VCF file

    Returns:
        Dictionary with Ti/Tv ratio and variant type counts
    """
    vcf_path = Path(vcf_path)

    if not vcf_path.exists():
        return {"error": "File not found"}

    # Run bcftools stats
    result = subprocess.run(
        ["bcftools", "stats", str(vcf_path)], capture_output=True, text=True
    )

    if result.returncode != 0:
        return {"error": "Failed to run bcftools stats", "stderr": result.stderr}

    stats_output = result.stdout
    lines = stats_output.split("\n")

    # Parse Ti/Tv ratio
    tstv_lines = [l for l in lines if l.startswith("TSTV")]
    titv_ratio = None
    n_transitions = 0
    n_transversions = 0

    if tstv_lines:
        fields = tstv_lines[0].split("\t")
        if len(fields) >= 5:
            n_transitions = int(fields[2])
            n_transversions = int(fields[3])
            titv_ratio = float(fields[4])

    # Parse SNP counts
    sn_lines = [l for l in lines if l.startswith("SN")]
    n_snps = 0
    n_indels = 0
    n_records = 0

    for line in sn_lines:
        if "number of SNPs:" in line:
            n_snps = int(line.split("\t")[-1])
        elif "number of indels:" in line:
            n_indels = int(line.split("\t")[-1])
        elif "number of records:" in line:
            n_records = int(line.split("\t")[-1])

    # Quality assessment
    quality_assessment = "unknown"
    if titv_ratio:
        if 2.8 <= titv_ratio <= 3.2:
            quality_assessment = "good_exome"
        elif 2.0 <= titv_ratio <= 2.1:
            quality_assessment = "good_wgs"
        elif titv_ratio < 2.0:
            quality_assessment = "low_quality"
        else:
            quality_assessment = "acceptable"

    return {
        "titv_ratio": titv_ratio,
        "n_transitions": n_transitions,
        "n_transversions": n_transversions,
        "n_snps": n_snps,
        "n_indels": n_indels,
        "n_records": n_records,
        "snp_fraction": n_snps / n_records if n_records > 0 else 0,
        "quality_assessment": quality_assessment,
    }


# Test
print("Testing compute_titv_stats()...")
result = compute_titv_stats(str(TOY_VCF))
print(json.dumps(result, indent=2))

Testing compute_titv_stats()...
{
  "titv_ratio": 1.65,
  "n_transitions": 5940,
  "n_transversions": 3607,
  "n_snps": 9511,
  "n_indels": 231,
  "n_records": 9745,
  "snp_fraction": 0.9759876859928168,
  "quality_assessment": "low_quality"
}


## Tool 2: Compute Sample-Level QC

**Note**: plink2 `--het` requires >=50 samples to estimate allele frequencies.  
For toy.vcf (3 samples), we'll skip heterozygosity and just compute missingness.

In [10]:
def compute_sample_qc(vcf_path: str, source: str = "unknown") -> dict:
    """
    Compute per-sample QC metrics using plink2.

    Args:
        vcf_path: Path to VCF file
        source: Data source for threshold selection

    Returns:
        Dictionary with sample QC results
    """
    vcf_path = Path(vcf_path)

    if not vcf_path.exists():
        return {"error": "File not found"}

    # Create temporary directory
    temp_dir = tempfile.mkdtemp(prefix="sample_qc_")
    temp_prefix = str(Path(temp_dir) / "plink")

    try:
        # Convert VCF to plink format
        print(f"Converting VCF to plink format...")
        result = subprocess.run(
            [
                "plink2",
                "--vcf",
                str(vcf_path),
                "--make-bed",
                "--out",
                temp_prefix,
                "--max-alleles",
                "2",
                "--allow-extra-chr",
                "--set-missing-var-ids",
                "@:#",
            ],
            capture_output=True,
            text=True,
        )

        if result.returncode != 0:
            return {"error": "plink2 conversion failed", "stderr": result.stderr}

        # Compute sample missingness
        print(f"Computing sample missingness...")
        result = subprocess.run(
            ["plink2", "--bfile", temp_prefix, "--missing", "--out", temp_prefix],
            capture_output=True,
            text=True,
        )

        if result.returncode != 0:
            return {"error": "plink2 missing failed", "stderr": result.stderr}

        # Read sample missingness
        smiss_file = f"{temp_prefix}.smiss"
        if not Path(smiss_file).exists():
            return {"error": "Missing file not created"}

        smiss = pd.read_csv(smiss_file, sep="\s+")

        # Source-specific thresholds
        thresholds = {
            "mayo_tapestry_helix": {"miss_thresh": 0.05},
            "mayo_biobank_regeneron": {"miss_thresh": 0.03},
            "ukbiobank": {"miss_thresh": 0.10},
            "1000genomes_phase3": {"miss_thresh": 0.15},
            "unknown": {"miss_thresh": 0.05},
        }

        thresh = thresholds.get(source, thresholds["unknown"])

        # Flag failing samples
        smiss["fail_missing"] = smiss["F_MISS"] > thresh["miss_thresh"]
        smiss["qc_pass"] = ~smiss["fail_missing"]

        # Summary
        summary = {
            "n_samples": len(smiss),
            "n_pass": int(smiss["qc_pass"].sum()),
            "n_fail": int((~smiss["qc_pass"]).sum()),
            "fail_samples": smiss[~smiss["qc_pass"]]["IID"].tolist(),
            "mean_missingness": float(smiss["F_MISS"].mean()),
            "std_missingness": float(smiss["F_MISS"].std()),
            "thresholds": thresh,
            "source": source,
            "note": "Heterozygosity skipped (requires >=50 samples)",
        }

        # Save detailed results
        qc_output_file = OUTPUT_DIR / "sample_qc.tsv"
        smiss.to_csv(qc_output_file, sep="\t", index=False)
        summary["detailed_results_file"] = str(qc_output_file)

        return summary

    finally:
        # Cleanup
        try:
            shutil.rmtree(temp_dir)
        except:
            pass


# Test
print("Testing compute_sample_qc()...")
result = compute_sample_qc(str(TOY_VCF), source="1000genomes_phase3")
print(json.dumps(result, indent=2))

Testing compute_sample_qc()...
Converting VCF to plink format...
Computing sample missingness...
{
  "n_samples": 3,
  "n_pass": 3,
  "n_fail": 0,
  "fail_samples": [],
  "mean_missingness": 0.0,
  "std_missingness": 0.0,
  "thresholds": {
    "miss_thresh": 0.15
  },
  "source": "1000genomes_phase3",
  "note": "Heterozygosity skipped (requires >=50 samples)",
  "detailed_results_file": "qc_output/sample_qc.tsv"
}


## Tool 3: Compute Variant-Level QC

In [11]:
def compute_variant_qc(vcf_path: str, source: str = "unknown") -> dict:
    """
    Compute per-variant QC metrics using plink2.

    Args:
        vcf_path: Path to VCF file
        source: Data source for threshold selection

    Returns:
        Dictionary with variant QC summary
    """
    vcf_path = Path(vcf_path)

    if not vcf_path.exists():
        return {"error": "File not found"}

    # Create temporary directory
    temp_dir = tempfile.mkdtemp(prefix="var_qc_")
    temp_prefix = str(Path(temp_dir) / "plink")

    try:
        # Convert VCF to plink
        print(f"Converting VCF to plink format...")
        result = subprocess.run(
            [
                "plink2",
                "--vcf",
                str(vcf_path),
                "--make-bed",
                "--out",
                temp_prefix,
                "--max-alleles",
                "2",
                "--allow-extra-chr",
                "--set-missing-var-ids",
                "@:#",
            ],
            capture_output=True,
            text=True,
        )

        if result.returncode != 0:
            return {"error": "plink2 conversion failed", "stderr": result.stderr}

        # Variant missingness (--missing generates .vmiss)
        print(f"Computing variant missingness...")
        result = subprocess.run(
            ["plink2", "--bfile", temp_prefix, "--missing", "--out", temp_prefix],
            capture_output=True,
            text=True,
        )

        if result.returncode != 0:
            return {"error": "plink2 missing failed", "stderr": result.stderr}

        vmiss = pd.read_csv(f"{temp_prefix}.vmiss", sep="\s+")

        # MAF
        print(f"Computing MAF...")
        result = subprocess.run(
            ["plink2", "--bfile", temp_prefix, "--freq", "--out", temp_prefix],
            capture_output=True,
            text=True,
        )

        if result.returncode != 0:
            return {"error": "plink2 freq failed", "stderr": result.stderr}

        freq = pd.read_csv(f"{temp_prefix}.afreq", sep="\s+")

        # Hardy-Weinberg
        print(f"Computing Hardy-Weinberg equilibrium...")
        result = subprocess.run(
            ["plink2", "--bfile", temp_prefix, "--hardy", "--out", temp_prefix],
            capture_output=True,
            text=True,
        )

        if result.returncode != 0:
            return {"error": "plink2 hardy failed", "stderr": result.stderr}

        hardy = pd.read_csv(f"{temp_prefix}.hardy", sep="\s+")

        # Merge all metrics
        var_qc = vmiss.merge(freq[["ID", "ALT_FREQS"]], on="ID", how="left")
        var_qc = var_qc.merge(hardy[["ID", "P"]], on="ID", how="left")
        var_qc = var_qc.rename(columns={"ALT_FREQS": "MAF", "P": "HWE_P"})

        # Calculate MAF (minor allele frequency)
        var_qc["MAF"] = var_qc["MAF"].apply(
            lambda x: min(float(x), 1 - float(x)) if pd.notna(x) else 0
        )

        # Source-specific filters
        filters = {
            "mayo_tapestry_helix": {"miss_max": 0.05, "hwe_min": 1e-6},
            "mayo_biobank_regeneron": {"miss_max": 0.03, "hwe_min": 1e-6},
            "ukbiobank": {"miss_max": 0.10, "hwe_min": 1e-8},
            "1000genomes_phase3": {"miss_max": 0.15, "hwe_min": 1e-8},
            "unknown": {"miss_max": 0.05, "hwe_min": 1e-6},
        }

        filt = filters.get(source, filters["unknown"])

        # Apply filters
        var_qc["pass"] = (var_qc["F_MISS"] < filt["miss_max"]) & (
            var_qc["HWE_P"] > filt["hwe_min"]
        )

        # Summary
        summary = {
            "n_variants": len(var_qc),
            "n_pass": int(var_qc["pass"].sum()),
            "n_fail": int((~var_qc["pass"]).sum()),
            "mean_missingness": float(var_qc["F_MISS"].mean()),
            "median_maf": float(var_qc["MAF"].median()),
            "n_rare_variants": int((var_qc["MAF"] < 0.01).sum()),
            "n_common_variants": int((var_qc["MAF"] >= 0.05).sum()),
            "n_hwe_fail": int((var_qc["HWE_P"] < filt["hwe_min"]).sum()),
            "filters": filt,
            "source": source,
        }

        # Save detailed results
        qc_output_file = OUTPUT_DIR / "variant_qc.tsv"
        var_qc.to_csv(qc_output_file, sep="\t", index=False)
        summary["detailed_results_file"] = str(qc_output_file)

        return summary

    finally:
        # Cleanup
        try:
            shutil.rmtree(temp_dir)
        except:
            pass


# Test
print("Testing compute_variant_qc()...")
result = compute_variant_qc(str(TOY_VCF), source="1000genomes_phase3")
print(json.dumps(result, indent=2))

Testing compute_variant_qc()...
Converting VCF to plink format...
Computing variant missingness...
Computing MAF...
Computing Hardy-Weinberg equilibrium...
{
  "n_variants": 9747,
  "n_pass": 9747,
  "n_fail": 0,
  "mean_missingness": 0.0,
  "median_maf": 0.0,
  "n_rare_variants": 9371,
  "n_common_variants": 376,
  "n_hwe_fail": 0,
  "filters": {
    "miss_max": 0.15,
    "hwe_min": 1e-08
  },
  "source": "1000genomes_phase3",
  "detailed_results_file": "qc_output/variant_qc.tsv"
}


## Tool 4: Generate QC Summary

In [12]:
def generate_qc_summary(
    titv_result: dict,
    sample_qc_result: dict,
    variant_qc_result: dict,
    source: str = "unknown",
) -> dict:
    """
    Generate comprehensive QC summary and pass/fail decision.
    """

    # Check for errors
    if "error" in titv_result:
        return {"status": "error", "component": "titv", "error": titv_result["error"]}
    if "error" in sample_qc_result:
        return {
            "status": "error",
            "component": "sample_qc",
            "error": sample_qc_result["error"],
        }
    if "error" in variant_qc_result:
        return {
            "status": "error",
            "component": "variant_qc",
            "error": variant_qc_result["error"],
        }

    # Validate required keys
    required_sample_keys = ["n_fail", "n_samples", "n_pass"]
    required_variant_keys = ["n_fail", "n_variants", "n_pass", "mean_missingness"]

    missing_sample = [k for k in required_sample_keys if k not in sample_qc_result]
    missing_variant = [k for k in required_variant_keys if k not in variant_qc_result]

    if missing_sample:
        return {
            "status": "error",
            "component": "sample_qc",
            "error": f"Missing required keys in sample_qc_result: {missing_sample}",
            "sample_qc_result": sample_qc_result,
        }

    if missing_variant:
        return {
            "status": "error",
            "component": "variant_qc",
            "error": f"Missing required keys in variant_qc_result: {missing_variant}",
            "variant_qc_result": variant_qc_result,
        }

    qc_checks = {}
    warnings = []
    failures = []

    # Check Ti/Tv
    titv = titv_result.get("titv_ratio")
    if titv:
        qc_checks["titv_ratio"] = titv
        if source == "1000genomes_phase3" or "wgs" in source.lower():
            if 2.0 <= titv <= 2.2:
                qc_checks["titv_pass"] = True
            else:
                qc_checks["titv_pass"] = False
                warnings.append(
                    f"Ti/Tv ratio {titv:.3f} outside expected WGS range (2.0-2.2)"
                )
        else:
            if 2.5 <= titv <= 3.5:
                qc_checks["titv_pass"] = True
            else:
                qc_checks["titv_pass"] = False
                failures.append(f"Ti/Tv ratio {titv:.3f} indicates quality issues")

    # Check sample failure rate
    sample_fail_rate = sample_qc_result["n_fail"] / sample_qc_result["n_samples"]
    qc_checks["sample_fail_rate"] = sample_fail_rate
    qc_checks["sample_qc_pass"] = sample_fail_rate <= 0.05

    if sample_fail_rate > 0.05:
        failures.append(
            f"{sample_fail_rate*100:.1f}% samples failed QC (threshold: 5%)"
        )

    # Check variant failure rate
    variant_fail_rate = variant_qc_result["n_fail"] / variant_qc_result["n_variants"]
    qc_checks["variant_fail_rate"] = variant_fail_rate
    qc_checks["variant_qc_pass"] = variant_fail_rate <= 0.20

    if variant_fail_rate > 0.20:
        failures.append(
            f"{variant_fail_rate*100:.1f}% variants failed QC (threshold: 20%)"
        )

    # Overall decision
    overall_pass = len(failures) == 0

    # Recommendations
    recommendations = []
    if overall_pass:
        recommendations.append("✓ PASS - Proceed to harmonization")
    else:
        recommendations.append("✗ FAIL - Manual review required")

    return {
        "status": "pass" if overall_pass else "fail",
        "source": source,
        "qc_checks": qc_checks,
        "summary": {
            "n_samples": sample_qc_result["n_samples"],
            "n_variants": variant_qc_result["n_variants"],
            "titv_ratio": titv,
            "sample_pass_rate": 1 - sample_fail_rate,
            "variant_pass_rate": 1 - variant_fail_rate,
        },
        "warnings": warnings,
        "failures": failures,
        "recommendations": recommendations,
    }


# Test it
print("Testing generate_qc_summary()...")
titv = {"titv_ratio": 1.65, "quality_assessment": "low_quality"}
sample = {"n_samples": 3, "n_pass": 3, "n_fail": 0, "mean_missingness": 0.0}
variant = {"n_variants": 9747, "n_pass": 9747, "n_fail": 0, "mean_missingness": 0.0}
result = generate_qc_summary(titv, sample, variant, "1000genomes_phase3")
print(json.dumps(result, indent=2))

Testing generate_qc_summary()...
{
  "status": "pass",
  "source": "1000genomes_phase3",
  "qc_checks": {
    "titv_ratio": 1.65,
    "titv_pass": false,
    "sample_fail_rate": 0.0,
    "sample_qc_pass": true,
    "variant_fail_rate": 0.0,
    "variant_qc_pass": true
  },
  "summary": {
    "n_samples": 3,
    "n_variants": 9747,
    "titv_ratio": 1.65,
    "sample_pass_rate": 1.0,
    "variant_pass_rate": 1.0
  },
  "warnings": [
    "Ti/Tv ratio 1.650 outside expected WGS range (2.0-2.2)"
  ],
  "failures": [],
  "recommendations": [
    "\u2713 PASS - Proceed to harmonization"
  ]
}


## Create QC Agent

In [ ]:
qc_agent = Agent(
    name="QC_Validation_Agent",
    model=LLM_MODEL,
    instruction="""
    You are a Genomics Quality Control Specialist.
    
    Execute comprehensive QC on VCF files using this EXACT workflow:
    
    STEP 1: Call compute_titv_stats(vcf_path)
      - Store the ENTIRE result dictionary as 'titv_result'
    
    STEP 2: Call compute_sample_qc(vcf_path, source)
      - Store the ENTIRE result dictionary as 'sample_result'
    
    STEP 3: Call compute_variant_qc(vcf_path, source)
      - Store the ENTIRE result dictionary as 'variant_result'
    
    STEP 4: Call generate_qc_summary(titv_result, sample_result, variant_result, source)
      - Pass the COMPLETE dictionaries from steps 1-3 as arguments
      - The function signature is: generate_qc_summary(titv_result: dict, sample_qc_result: dict, variant_qc_result: dict, source: str)
      - You must pass the FULL result objects, not summaries or subsets
    
    IMPORTANT: 
    - Do NOT summarize or extract fields from the tool results before passing them to generate_qc_summary
    - Pass the EXACT dict objects returned by each tool
    - All four tool calls are required
    
    Report findings clearly with specific metrics.
    Flag concerns with ⚠️, failures with ✗, passes with ✓.
    """,
    tools=[
        compute_titv_stats,
        compute_sample_qc,
        compute_variant_qc,
        generate_qc_summary,
    ],
)

print("✓ QC Agent created")

## Test QC Agent

In [14]:
runner = InMemoryRunner(agent=qc_agent)

query = f"""
Perform comprehensive QC on: {TOY_VCF}
Source: 1000genomes_phase3

Run all QC steps and provide detailed findings.
Note: Only 3 samples, so heterozygosity metrics are limited.
"""

print("🔬 Starting QC Analysis")
print("=" * 80)
response = await runner.run_debug(query)
print("=" * 80)
print("✅ QC Complete")

🔬 Starting QC Analysis

 ### Created new session: debug_session_id

User > 
Perform comprehensive QC on: data/toy.vcf
Source: 1000genomes_phase3

Run all QC steps and provide detailed findings.
Note: Only 3 samples, so heterozygosity metrics are limited.



QC_Validation_Agent > I will start by computing the Ti/Tv ratio, followed by sample QC and variant QC. Finally, I will generate a QC summary report.

Converting VCF to plink format...
Computing sample missingness...
Converting VCF to plink format...
Computing variant missingness...
Computing MAF...
Computing Hardy-Weinberg equilibrium...
QC_Validation_Agent > There was an error generating the QC summary. The `sample_qc_result` dictionary is missing the required keys: `n_fail`, `n_samples`, and `n_pass`. Please ensure these keys are present in the dictionary before proceeding.
✅ QC Complete
